### Гипотезы
1. Убрать ненужные слова по типу: расскажи, докажи, из, ...

    Есть два пути: выбрать опорные слова из определений и оставить только их или собрать служебные и второстепенные слова и выкинуть служебные
2. На каком языке лучше английском или русском? (кажется, что модель вообще не воспринимает русский и проще дообучить). Что лучше: улучшить перевод или дообучить модель?
3. Дообучить модель на корпусте текста (учебник мб)
4. Взять другую, русскую модель и использовать

### Как считаю метрику?
Беру два вопроса: пользовательский и из базы знаний, привожу их к одному виду (прописные буквы, начальная форма, удаление стоп слов), потом считаю пересечение и делю на количество слов в вопросе из БЗ.

**Улчушение**: для каждого слова определить его важность: как количество вхождений в БЗ, либо ручками составить словарь с важными терминами.

In [1]:
!pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 83.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=717f72e37f547eba124c6d4861e622e7341d547df8737bad7e90b148384f6624
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
import re
import nltk
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import pymorphy2
from collections import Counter

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

stopwords_ru = stopwords.words('russian')

tokenizer = RegexpTokenizer('\w+')

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [28]:
def make_beautiful(text):
    text = text.lower() # единый формат
    text = re.sub('\n|\t|\r', ' ', text) # убираем лишнее
    sents = sent_tokenize(text) # разбиваем на предложения
    tokenizer = RegexpTokenizer('\w+')
    sents_tokenize = [tokenizer.tokenize(item) for item in sents] # разбиваем на слова
    sents_tokenize = [[item for item in sent if item not in stopwords_ru] for sent in sents_tokenize] # лемматизируйте все слова из датасета
    sents_tokenize = [[morph.normal_forms(item)[0] for item in sent] for sent in sents_tokenize] # лемматизируйте все слова из датасета
    words = [item for sent in sents_tokenize for item in sent] # получаем обработанные слова

    return words

In [29]:
def calculate_k(user, database):
    in_both = list(set(user) & set(database))
    k = len(in_both) / len(database)

    return k

In [30]:
question = make_beautiful('Как можно вычислить частичные производные Антимирова?')

In [31]:
user_question = make_beautiful('че такое эти ваши производные антимирова')

In [32]:
K = calculate_k(user_question, question)

In [33]:
K

0.5

In [34]:
word_dict = Counter(question)
word_dict.most_common()[:20]
words = word_dict.most_common()

In [35]:
words

[('вычислить', 1), ('частичный', 1), ('производный', 1), ('антимиров', 1)]